<a href="https://colab.research.google.com/github/Lelouchlamperougexd/article_experiment/blob/main/new_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install datasets pandas numpy torch tqdm faiss-cpu sentence-transformers transformers openpyxl kaggle

# Import necessary libraries
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import faiss
from sentence_transformers import SentenceTransformer
import json
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Set up Kaggle API and download dataset
print("Downloading ArXiv dataset from Kaggle...")
os.environ["KAGGLE_CONFIG_DIR"] = "/content"

# Download dataset
!kaggle datasets download -d cornell-university/arxiv --unzip

# Path to dataset file
FILE_PATH = "arxiv-metadata-oai-snapshot.json"

Dataset URL: https://www.kaggle.com/datasets/cornell-university/arxiv
License(s): CC0-1.0
 98% 1.38G/1.40G [00:08<00:00, 202MB/s]
100% 1.40G/1.40G [00:08<00:00, 180MB/s]


In [3]:
# Load dataset from JSON
print("Loading ArXiv dataset...")
try:
    with open(FILE_PATH, "r") as f:
        arxiv_data = [json.loads(line) for line in f]

    docs_arxiv_df = pd.DataFrame(arxiv_data)

    # Keep only necessary columns
    docs_arxiv_df = docs_arxiv_df[['id', 'title', 'abstract', 'categories', 'update_date']]
    docs_arxiv_df = docs_arxiv_df.rename(columns={'id': 'doc_id'})

    # Process full text
    docs_arxiv_df['full_text'] = docs_arxiv_df['title'] + ". " + docs_arxiv_df['abstract'].fillna("")

    # Convert date
    docs_arxiv_df['update_date'] = pd.to_datetime(docs_arxiv_df['update_date'], errors='coerce')

    # Remove NaT dates
    docs_arxiv_df = docs_arxiv_df.dropna(subset=['update_date'])

    # Split into old (before 2020) and new (2021+)
    old_docs_df = docs_arxiv_df[docs_arxiv_df['update_date'] < '2020-01-01']
    new_docs_df = docs_arxiv_df[docs_arxiv_df['update_date'] >= '2021-01-01']

    print(f"Loaded {len(docs_arxiv_df)} total documents.")
    print(f"Filtered {len(old_docs_df)} old papers and {len(new_docs_df)} new papers.")
except Exception as e:
    print(f"Error loading dataset: {e}")
    old_docs_df = new_docs_df = pd.DataFrame()


Loading ArXiv dataset...
Loaded 2677526 total documents.
Filtered 1536774 old papers and 966884 new papers.


In [ ]:
# Create FAISS index for old papers
print("Creating FAISS index for old papers...")
model = SentenceTransformer('all-MiniLM-L6-v2')  # Compact transformer model
old_texts = old_docs_df['full_text'].tolist()
old_doc_ids = old_docs_df['doc_id'].tolist()

# Encode documents into embeddings
old_embeddings = model.encode(old_texts, convert_to_tensor=True)
old_embeddings_np = old_embeddings.cpu().numpy()

# Normalize for cosine similarity
faiss.normalize_L2(old_embeddings_np)
index = faiss.IndexFlatIP(old_embeddings_np.shape[1])
index.add(old_embeddings_np)

Creating FAISS index for old papers...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Function to determine novelty

def evaluate_novelty(new_text, top_k=5):
    """Evaluate novelty of a new article by comparing with old papers."""
    new_embedding = model.encode([new_text], convert_to_tensor=True).cpu().numpy()
    faiss.normalize_L2(new_embedding)
    scores, indices = index.search(new_embedding, k=top_k)
    avg_similarity = np.mean(scores)
    novelty_score = 1 - avg_similarity  # Higher score means more novel
    return novelty_score

In [ ]:
# Run novelty evaluation
print("Evaluating novelty of new papers...")
novelty_results = []
for _, row in tqdm(new_docs_df.iterrows(), total=len(new_docs_df)):
    novelty_score = evaluate_novelty(row['full_text'])
    novelty_results.append({'doc_id': row['doc_id'], 'title': row['title'], 'novelty_score': novelty_score})

In [ ]:
# Convert to DataFrame and sort by novelty
novelty_df = pd.DataFrame(novelty_results).sort_values(by='novelty_score', ascending=False)
print("Top 10 most novel papers:")
print(novelty_df.head(10))

In [ ]:
# Save results
novelty_df.to_csv("novelty_results.csv", index=False)
print("Saved novelty results to novelty_results.csv")